In [ ]:
!pip install torch
!pip install pytorch-pretrained-bert
!pip install sentence-transformers
!pip install Rouge
!pip install dataset
!pip install evaluate

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

# Bert And Kmeans Code 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def bertSent_embeding(sentences):
    # Add sentence head and tail as BERT requested
    marked_sent = ["[CLS] " +item + " [SEP]" for item in sentences]
    tokenized_sent = [tokenizer.tokenize(item ) for item in marked_sent]
    # index to BERT vocabulary
    indexed_tokens = [tokenizer.convert_tokens_to_ids(item) for item in tokenized_sent]
    tokens_tensor = [torch.tensor([item]) for item in indexed_tokens]
    # add segment id as BERT requested
    segments_ids = [[1] * len(item) for ind,item in enumerate(tokenized_sent)]
    segments_tensors = [torch.tensor([item]) for item in segments_ids]
    # load BERT base model and set to evaluation mode
    ##### bert_model = SentenceTransformer('stsb-bert-large')
    #model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. 
    #For example, Dropouts Layers, BatchNorm Layers etc.
    #You need to turn them off during model evaluation, and .eval() will do it for you. In addition, 
    #the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation:
    bert_model.eval()
    ## Output 12 layers of latent vector
    assert len(tokens_tensor) == len(segments_tensors)
    encoded_layers_list = []
    for i in range(len(tokens_tensor)):
        with torch.no_grad():
            encoded_layers, _ = bert_model(tokens_tensor[i], segments_tensors[i])
        encoded_layers_list.append(encoded_layers)
    ## Use only the last layer vetcor **we can use others**
    token_vecs_list =[layers[11][0] for layers in encoded_layers_list]
    ## Pooling word vector to sentence vector, use mean pooling
    sentence_embedding_list = [torch.mean(vec, dim=0).numpy() for vec in token_vecs_list]
    return sentence_embedding_list

100%|██████████| 407873900/407873900 [00:24<00:00, 16416816.96B/s]


In [ ]:
def kmeans_sumIndex(sentence_embedding_list):
  try:
    n_clusters =5
    #np.ceil(len(sentence_embedding_list)**0.5)
    kmeans = KMeans(n_clusters=int(n_clusters))
    kmeans = kmeans.fit(sentence_embedding_list)
    #Compute minimum distances between one point and a set of points.
    #get centers and compare with sentence_embedding by bert !!!!("centers is the most common sentences")
    sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
    sum_index = sorted(sum_index)
    return sum_index
  except:
    n_clusters =1
    #np.ceil(len(sentence_embedding_list)**0.5)
    kmeans = KMeans(n_clusters=int(n_clusters))
    kmeans = kmeans.fit(sentence_embedding_list)
    #Compute minimum distances between one point and a set of points.
    #get centers and compare with sentence_embedding by bert !!!!("centers is the most common sentences")
    sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
    sum_index = sorted(sum_index)
    return sum_index
''' n_clusters =np.ceil(len(sentence_embedding_list)**0.5)
kmeans = KMeans(n_clusters=int(n_clusters))
kmeans = kmeans.fit(sentence_embedding_list)
#Compute minimum distances between one point and a set of points.
#get centers and compare with sentence_embedding by bert ("centers is the most common sentences")
sum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric='euclidean')
sum_index = sorted(sum_index)
return sum_index'''

' n_clusters =np.ceil(len(sentence_embedding_list)**0.5)\nkmeans = KMeans(n_clusters=int(n_clusters))\nkmeans = kmeans.fit(sentence_embedding_list)\n#Compute minimum distances between one point and a set of points.\n#get centers and compare with sentence_embedding by bert ("centers is the most common sentences")\nsum_index,_ = pairwise_distances_argmin_min(kmeans.cluster_centers_, sentence_embedding_list,metric=\'euclidean\')\nsum_index = sorted(sum_index)\nreturn sum_index'

In [ ]:
def bertSummarize(text):
  sentences = sent_tokenize(text)
  sentence_embedding_list = bertSent_embeding(sentences)
  sum_index = kmeans_sumIndex(sentence_embedding_list)
  summary = ''.join([sentences[ind] for ind in sum_index])
  return summary

# DataLoad And PreProcess
**this data has changed to  cnn-dailymail**

In [ ]:
Data_Frame_Train_inputs =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/train-inputs.csv") #Cochrane train
Data_Frame_Train_targets =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/train-targets.csv") #Cochrane train
#------------------------------------------------------------------------------------------------------------------------------------------------
#Data_Frame_val_inputs =pd.read_json("https://drive.google.com/u/0/uc?id=1-FmSaBFQPB3Det_t6VHeBJjsJKVEBYRq&export=download") #Cochrane validation

#Data_Frame_val_inputs =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/dev-inputs.csv") #Cochrane validation
#Data_Frame_val_targets =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/dev-targets.csv") #Cochrane validation
#------------------------------------------------------------------------------------------------------------------------------------------------
#Data_Frame_Test_inputs =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/dev-inputs.csv") #Cochrane test
#Data_Frame_Test_targets =pd.read_csv("https://github.com/bwallace/RCT-summarization-data/raw/main/dev-targets.csv") #Cochrane test

In [ ]:
Train_Inputs = Data_Frame_Train_inputs[['ReviewID','Title','Abstract']]
Train_Targets = Data_Frame_Train_targets[['ReviewID','Target']]

for i in Train_Inputs.index:
  val =  Train_Inputs.at[i,"Abstract"]
  if val != val:
    val = Train_Inputs.at[i,"Title"]
  Train_Inputs.at[i,"Abstract"] = val

#Train_Inputs = Train_Inputs.dropna()
#Train_Targets =Train_Targets.dropna()
#------------------------------------------------------------------------------
'''val_Inputs = Data_Frame_val_inputs[['ReviewID','Title','Abstract']]
val_Targets = Data_Frame_val_targets[['ReviewID','Target']]

val_Inputs['REVIEWS'] = val_Inputs['Title'] + ' ' +val_Inputs['Abstract']#combine abstract and title

val_Inputs = val_Inputs.dropna()
val_Targets =val_Targets.dropna()'''
#------------------------------------------------------------------------------
'''test_Inputs = Data_Frame_val_inputs[['ReviewID','Title','Abstract']]
test_Targets = Data_Frame_val_targets[['ReviewID','Target']]

test_Inputs = test_Inputs.dropna()
val_Targets = val_Targets.dropna()
'''

"test_Inputs = Data_Frame_val_inputs[['ReviewID','Title','Abstract']]\ntest_Targets = Data_Frame_val_targets[['ReviewID','Target']]\n\ntest_Inputs = test_Inputs.dropna()\nval_Targets = val_Targets.dropna()\n"

In [ ]:
Train_Inputs.head(18)

,ReviewID,Title,Abstract
0,CD007697,Aggressive surgical effort and improved surviv...,Residual disease after initial surgery for ova...
1,CD000174,Prophylactic indomethacin therapy in the first...,To determine whether a course of low-dose indo...
2,CD000174,Indomethacin reduces the risks of severe intra...,"A prospective, random selection, double-blind ..."
3,CD000174,Administration of indomethacin for the prevent...,One hundred twenty-two preterm infants were en...
4,CD000174,Early intravenous indomethacin prolongs respir...,Infants weighing 1500 g at birth requiring eit...
5,CD000174,Low-dose indomethacin therapy and extension of...,We enrolled 61 neonates of 600 to 1250 gm birt...
6,CD000174,Long-term effects of indomethacin prophylaxis ...,The prophylactic administration of indomethaci...
7,CD000174,Randomized low-dose indomethacin trial for pre...,We admitted 36 preterm neonates (600 to 1250 g...
8,CD000174,Prophylactic indomethacin for prevention of in...,The impact of early prophylactic use of intrav...
9,CD000174,Effects of early indomethacin administration o...,A previous study found that early intravenous ...


In [ ]:
validation_inputs_cleared = val_Inputs.drop_duplicates(subset=['ReviewID'])

In [ ]:
print(len(Train_Inputs ))

40497


# **Combine matched reviews in id** 

In [ ]:
for key,ID in validation_inputs_cleared['ReviewID'].items():
  for KEY,id in val_Inputs['ReviewID'].items():
    if ID == id :
      if val_Inputs['REVIEWS'][KEY] not in validation_inputs_cleared['REVIEWS'][key]:
        validation_inputs_cleared['REVIEWS'][key] = validation_inputs_cleared['REVIEWS'][key] + val_Inputs['REVIEWS'][KEY]

<ipython-input-15-dcfee36aad03>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_inputs_cleared['REVIEWS'][key] = validation_inputs_cleared['REVIEWS'][key] + val_Inputs['REVIEWS'][KEY]


In [ ]:
validation_inputs_cleared.head(3)

# **Extraction step**

In [ ]:
Train_Inputs.index

RangeIndex(start=0, stop=40497, step=1)

In [ ]:
for i in Train_Inputs.index:
  Train_Inputs.at[i,'Abstract']  = bertSummarize(Train_Inputs.at[i,'Abstract'])

key 0 



/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-16-1f71e1592461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Extracted_sent['Abstract'][key] = bertSummarize(Train_Inputs['Abstract'][key])


key 1 



/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-16-1f71e1592461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Extracted_sent['Abstract'][key] = bertSummarize(Train_Inputs['Abstract'][key])


key 2 



...

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-16-1f71e1592461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Extracted_sent['Abstract'][key] = bertSummarize(Train_Inputs['Abstract'][key])


key 13 



/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-16-1f71e1592461>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Extracted_sent['Abstract'][key] = bertSummarize(Train_Inputs['Abstract'][key])


key 14 



KeyboardInterrupt: ignored

**Save Dataframe into csv file to be ready to the next step**

In [ ]:
Extracted_sent.to_csv(r'/content/FinalA1_100.csv')
Extracted_sent.head(5)


In [ ]:
for i in Data_Frame_val_inputs['Text']:
  print(i)

In [ ]:
Data_Frame_val_inputs['Text'][0:500]

0      In a combinatorial auction m heterogenous indi...
1      We consider the free-rider problem in peer-to-...
2      Determining how to transport delay-sensitive v...
3      Botnets pose a serious threat to the health of...
4      Under many protocols-in computerized settings ...
                             ...                        
495    Mobile Online Social Networks (mOSNs) have rec...
496    A recent trend in optimizing Internet browsing...
497    System administrators have utilized log analys...
498    Continuous fine-grain status monitoring of a c...
499    Highly interactive web applications that offer...
Name: Text, Length: 500, dtype: object

In [ ]:
print(''.join(str(v) for v in Data_Frame_val_inputs['Text']))

In [ ]:
Data_Frame_val_inputs['Text'][0]

'In a combinatorial auction m heterogenous indivisible items are sold to n bidders.\nThis paper considers settings in which the valuation functions of the bidders are known to be complement-free (a.k.a. subadditive).\nWe provide several approximation algorithms for the social-welfare maximization problem in such settings.\nFirstly, we present a logarithmic upper bound for the case that the access to the valuation functions is via demand queries.\nFor the weaker value queries model we provide a tight O(√ m) approximation.\nUnlike the other algorithms we present, this algorithm is also incentive compatible.\nFinally, we present two approximation algorithms for the more restricted class of XOS valuations: A simple deterministic algorithm that provides an approximation ratio of 2 and an optimal e e−1 approximation achieved via randomized rounding.\nWe also present optimal lower bounds for both the demand oracles model and the value oracles model.\nCF valuations to within a factor of 2 − , 

In [ ]:
Extracted_sent=[]

Extracted_sent.append(bertSummarize(Data_Frame_val_inputs['Text'][0]))


100%|██████████| 407873900/407873900 [00:16<00:00, 24582690.34B/s]
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
''.join(str(v) for v in Extracted_sent)

'[?]\n. The best deterministic mechanism for complement-free valuations is still the O( √ m)-mechanism presented in this paper.Dobzinski and Nisan [5] proved that one cannot achieve a much better approximation factor using this type of algorithms -maximal in range algorithms (see the discussion in Section 5).1.5 Open Questions. Therefore, if all valuations are submodular, the algorithm presented in this section requires demand queries only (recall that a value query can be simulated by a polynomial number of demand queries [2]). However, when one attempts randomized rounding on packing problems such as combinatorial auctions the results are not good; A randomized choice will very likely yield non-feasible solutions, unless the probabilities chosen reduce the expected quality of solution by a large O( √ m) factor.Both algorithms we present in this section start with a randomized rounding procedure for obtaining a "pre-allocation". Theorem 3.1 If all input valuations are complement-free 

In [ ]:
''.join(str(v) for v in Data_Frame_val_inputs['Target'][0])

'Finally, we present two approximation algorithms for the more restricted class of XOS valuations: A simple deterministic algorithm that provides an approximation ratio of 2 and an optimal e e−1 approximation achieved via randomized rounding.\nWe also present optimal lower bounds for both the demand oracles model and the value oracles model.\nThis approximation ratio may seem quite bad when contrasted with the fact that for the class of submodular valuations constant-approximation algorithms that use only value queries exist (e.g., by Lehmann et al. [12]).\nHowever, it turns out that this algorithm is optimal in the value oracle model, even if all bidders have XOS valuations, and even ignoring truthfulness constraints: Theorem 1.6 Every approximation algorithm for combinatorial auctions with XOS bidders that uses only value queries, requires an exponential number of queries to achieve an approximation ratio better than m 1 2 − , for any constant > 0.1.4 Subsequent Work.\nKhot, Lipton, 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
from rouge import Rouge
rouge = Rouge()
print(rouge.get_scores(''.join(str(v) for v in Extracted_sent), ''.join(str(v) for v in Data_Frame_val_inputs['Target'][0])))

[{'rouge-1': {'r': 0.34202898550724636, 'p': 0.8939393939393939, 'f': 0.4947589058502433}, 'rouge-2': {'r': 0.19689119170984457, 'p': 0.7215189873417721, 'f': 0.3093622761430789}, 'rouge-l': {'r': 0.3391304347826087, 'p': 0.8863636363636364, 'f': 0.4905660337328429}}]


In [ ]:
Data_Frameout=pd.read_csv("/content/Final.csv")

In [ ]:
print(len(Data_Frameout))

3752


In [ ]:
validation_inputs_cleared = Data_Frameout.drop_duplicates(subset=['ReviewID'])

In [ ]:
print(len(validation_inputs_cleared))

In [ ]:
for key,ID in validation_inputs_cleared['ReviewID'].items():
  for KEY,id in validation_inputs_cleared['ReviewID'].items():
    if ID == id :
      if validation_inputs_cleared['REVIEWS'][KEY] not in validation_inputs_cleared['REVIEWS'][key]:
        validation_inputs_cleared['REVIEWS'][key] = validation_inputs_cleared['REVIEWS'][key] + validation_inputs_cleared['REVIEWS'][KEY]

# Abstractive Section

start training